pip install keras

pip install tensorflow

pip install flask

pip install sklearn

In [ ]:
import os
from flask import Flask, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename
app = Flask(__name__)
from keras.models import load_model
from keras import backend as K
from tensorflow.compat.v1.keras import backend as K
from skimage.transform import resize
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
print("Loading model")
# set the session
global sess
sess = tf.compat.v1.Session()
K.set_session(sess)
# to get graph
global graph
graph = tf.compat.v1.get_default_graph()
# GET and POST for the web
@app.route('/', methods=['GET','POST'])

def main_page():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        file.save(os.path.join('uploads', filename))
        return redirect(url_for('prediction', filename=filename))
    return render_template('service.html')
@app.route('/prediction/<filename>')
def prediction(filename):
    #Step 1
    my_image = plt.imread(os.path.join('uploads', filename))
    #Step 2
    my_image_re = resize(my_image, (224,224,3))
    
    #Step 3
    with graph.as_default():
        K.set_session(sess)
        model = load_model('leaf-cnn.h5')
        probabilities = model.predict(np.array( [my_image_re,] ))[0,:]
        print(probabilities)
#Step 4
    number_to_class = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy']
      
    index = np.argsort(probabilities)
       
    predictions = {
       "class1":number_to_class[index[3]],
        "class2":number_to_class[index[2]],
        "prob1":probabilities[index[3]],
        "prob2":probabilities[index[2]],
      }
#Step 5
    return render_template('predict.html', predictions=predictions)
app.debug = False
app.run(host='localhost')